<a href="https://colab.research.google.com/github/Pitap4ik/UkrainianPremierLeague_Analytics/blob/main/UkrainianPremierLeague_DataScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import requests

In [29]:
standings_url = "https://fbref.com/en/comps/39/Ukrainian-Premier-League-Stats"

In [30]:
data = requests.get(standings_url)

In [31]:
from bs4 import BeautifulSoup

In [32]:
soup = BeautifulSoup(data.text)

In [33]:
standings_table = soup.select('table.stats_table')[0]

In [34]:
links = standings_table.find_all('a')

In [35]:
links = [l.get("href") for l in links]

In [36]:
links = [l for l in links if 'squads' in l]

In [37]:
team_urls = [f"https://Fbref.com{l}" for l in links]

In [38]:
team_url = team_urls[0]

In [39]:
data=requests.get(team_url)

In [40]:
import pandas as pd
from io import StringIO

In [41]:
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")

In [42]:
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-07-23,21:00,Champions Lg,Second qualifying round,Tue,Home,W,6.0,2.0,rs Partizan,NaN,NaN,NaN,4712.0,Vitaliy Buyalskyi,4-3-3,4-3-3,Christian Dingert,Match Report,Leg 1 of 2
1,2024-07-31,20:00,Champions Lg,Second qualifying round,Wed,Away,W,3.0,0.0,rs Partizan,NaN,NaN,NaN,10.0,Vitaliy Buyalskyi,4-3-3,4-2-3-1,Marian Barbu,Match Report,Leg 2 of 2; Dynamo Kyiv won
2,2024-08-06,20:00,Champions Lg,Third qualifying round,Tue,Home,D,1.0,1.0,sct Rangers,NaN,NaN,NaN,8315.0,Andriy Yarmolenko,4-3-3,4-4-2,Donatas Rumšas,Match Report,Leg 1 of 2
3,2024-08-09,18:00,Premier League,Matchweek 2,Fri,Away,W,2.0,1.0,Veres Rivne,NaN,NaN,NaN,4509.0,Vitaliy Buyalskyi,4-2-3-1,4-4-2,NaN,Match Report,NaN
4,2024-08-13,19:45,Champions Lg,Third qualifying round,Tue,Away,W,2.0,0.0,sct Rangers,NaN,NaN,NaN,39180.0,Vitaliy Buyalskyi,4-3-3,4-2-3-1,Marco Guida,Match Report,Leg 2 of 2; Dynamo Kyiv won


In [43]:
soup=BeautifulSoup(data.text)

In [44]:
links=soup.find_all('a')

In [45]:
links=[l.get("href") for l in links]

In [46]:
links=[l for l in links if l and "all_comps/shooting/" in l]

In [47]:
shooting_url=f"https://FBref.com{links[0]}"

In [48]:
data=requests.get(shooting_url)

In [49]:
shootings =pd.read_html(StringIO(data.text),match = "Shooting")

In [50]:
shooting = shootings[0]

In [51]:
shooting.head()

For Dynamo Kyiv                                                             \
             Date   Time            Comp                    Round  Day Venue   
0      2024-07-23  21:00    Champions Lg  Second qualifying round  Tue  Home   
1      2024-07-31  20:00    Champions Lg  Second qualifying round  Wed  Away   
2      2024-08-06  20:00    Champions Lg   Third qualifying round  Tue  Home   
3      2024-08-09  18:00  Premier League              Matchweek 2  Fri  Away   
4      2024-08-13  19:45    Champions Lg   Third qualifying round  Tue  Away   

                                 ... Standard              Expected       \
  Result   GF   GA     Opponent  ...     Dist  FK PK PKatt       xG npxG   
0      W  6.0  2.0  rs Partizan  ...      NaN NaN  0     0      NaN  NaN   
1      W  3.0  0.0  rs Partizan  ...      NaN NaN  1     1      NaN  NaN   
2      D  1.0  1.0  sct Rangers  ...      NaN NaN  0     0      NaN  NaN   
3      W  2.0  1.0  Veres Rivne  ...      NaN NaN  0     0      NaN  NaN   
4      W  2.0  0.0  sct Rangers  ...      NaN NaN  0     0      NaN  NaN   

                       Unnamed: 25_level_0  
  npxG/Sh G-xG np:G-xG        Match Report  
0     NaN  NaN     NaN        Match Report  
1     NaN  NaN     NaN        Match Report  
2     NaN  NaN     NaN        Match Report  
3     NaN  NaN     NaN        Match Report  
4     NaN  NaN     NaN        Match Report  

[5 rows x 26 columns]

In [52]:
shooting.columns = shooting.columns.droplevel()

In [53]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-07-23,21:00,Champions Lg,Second qualifying round,Tue,Home,W,6.0,2.0,rs Partizan,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2024-07-31,20:00,Champions Lg,Second qualifying round,Wed,Away,W,3.0,0.0,rs Partizan,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Match Report
2,2024-08-06,20:00,Champions Lg,Third qualifying round,Tue,Home,D,1.0,1.0,sct Rangers,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2024-08-09,18:00,Premier League,Matchweek 2,Fri,Away,W,2.0,1.0,Veres Rivne,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
4,2024-08-13,19:45,Champions Lg,Third qualifying round,Tue,Away,W,2.0,0.0,sct Rangers,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report


In [62]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on = "Date")

In [63]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-07-23,21:00,Champions Lg,Second qualifying round,Tue,Home,W,6.0,2.0,rs Partizan,...,4-3-3,Christian Dingert,Match Report,Leg 1 of 2,NaN,NaN,NaN,NaN,0,0
1,2024-07-31,20:00,Champions Lg,Second qualifying round,Wed,Away,W,3.0,0.0,rs Partizan,...,4-2-3-1,Marian Barbu,Match Report,Leg 2 of 2; Dynamo Kyiv won,NaN,NaN,NaN,NaN,1,1
2,2024-08-06,20:00,Champions Lg,Third qualifying round,Tue,Home,D,1.0,1.0,sct Rangers,...,4-4-2,Donatas Rumšas,Match Report,Leg 1 of 2,NaN,NaN,NaN,NaN,0,0
3,2024-08-09,18:00,Premier League,Matchweek 2,Fri,Away,W,2.0,1.0,Veres Rivne,...,4-4-2,NaN,Match Report,NaN,NaN,NaN,NaN,NaN,0,0
4,2024-08-13,19:45,Champions Lg,Third qualifying round,Tue,Away,W,2.0,0.0,sct Rangers,...,4-2-3-1,Marco Guida,Match Report,Leg 2 of 2; Dynamo Kyiv won,NaN,NaN,NaN,NaN,0,0


In [72]:
years=list(range(2025,2023,-1))

In [74]:
years

[2025, 2024]

In [75]:
standings_url = "https://fbref.com/en/comps/39/Ukrainian-Premier-League-Stats"

In [76]:
import time
all_matches = []
for year in years:

  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get('href') for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace('-Stats','').replace('-',' ')

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match='Scores & Fixtures')[0]

    soup = BeautifulSoup(data.text)
    links = [l.get('href') for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on='Date')
    except ValueError:
      continue

    team_data = team_data[team_data['Comp'] == 'Premier League']
    team_data['Season'] = year
    team_data['Team'] = team_name
    all_matches.append(team_data)
    time.sleep(1)

SyntaxError: incomplete input (<ipython-input-76-7fbe9f4e00de>, line 23)

In [ ]:
match_df = pd.concat(all_matches)